In [418]:
import pandas as pd
from connect import PostgreSQL
from tqdm import tqdm
import datetime as dt
from datetime import date
import plotly.graph_objs as go

# Parâmetros

In [419]:
_schema = 'H-1'

In [420]:
_pixels = 720

pixels_width = int((_pixels/9)*16)
pixels_height = int(_pixels)

In [421]:
date_query = date(int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[0])), int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[1])), 1).strftime('%Y-%m-%d')

In [422]:
codigos_kaizen = [('KAIZEN ASA NORTE', '13996'),
                  ('KAIZEN CEILÂNDIA', '16100'),
                  (     'KAIZEN GAMA', '18400'),
                  (  'KAIZEN SOF SUL', '20000')]

In [423]:
lista_lojas = (['01',             'CD'],
               ['02',               ''],
               ['03',  '04. ASA NORTE'],
               ['04',  '02. CEILÂNDIA'],
               ['05',       '03. GAMA'],
               ['06',    '01. SOF SUL'],
               ['VO', '05. E-COMMERCE'])

In [424]:
relatorio = 'ACOMPANHAMENTO DE VENDA'

relatorio_lower = relatorio.lower()
relatorio_msg = relatorio_lower.replace(relatorio_lower.split()[0], relatorio_lower.split()[0].title())

In [425]:
caminho_imagem = 'Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\REPORT'

In [426]:
contatos = [('PrimeiraFoto', 'https://chat.whatsapp.com/LK4LDajkH52BNydoLGGD2d', True),
            ('Dados e Int',  'https://chat.whatsapp.com/JxY14kWarG70MxB2jzb5pc', False),
            ('Dream Team',   'https://chat.whatsapp.com/Hb0KMgYnuFB8coTPlFKTzJ', False),
            ('Report Di',    'https://chat.whatsapp.com/G4p8qdUIUblCNPS80244RQ', False),
            ('API',          'https://chat.whatsapp.com/EBLX0fxYfvuHL7jZ9CZECQ', True),
            ('Teste Python', 'https://chat.whatsapp.com/I4y7EAVfns629NvfoLUtGI', True)]

#

# DataFrames

## Clientes

In [427]:
df_cliente = PostgreSQL.read_postgres(name_table='cliente', schema=_schema)

In [428]:
df_cli_ped = PostgreSQL.read_postgres(name_table='cli_ped', schema=_schema, query=f'WHERE DT_CADASTR >= \'{date_query}\'')

## Pedidos

In [429]:
df_pedido = PostgreSQL.read_postgres(name_table='pedido', schema=_schema, query=f'WHERE DT_EMISSAO >= \'{date_query}\'')

In [430]:
df_pedido['VALOR_TOT'] = df_pedido['VALOR_TOT'].astype(float)

In [431]:
df_pedido = df_pedido[(df_pedido['CODCLI'] != '13996') &
                      (df_pedido['CODCLI'] != '16100') &
                      (df_pedido['CODCLI'] != '18400') &
                      (df_pedido['CODCLI'] != '20000') &
                      (df_pedido['CODVDE'] != '0100') &
                      (df_pedido['CODVDE'] != '0001') &
                      (df_pedido['TIPPED'] == 'V')]

In [432]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'FORMA_PGTO'] == 'M':
        df_pedido.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_pedido.loc[i, 'COD_LOJA'] = df_pedido.loc[i, 'CD_LOJA']

100%|██████████| 2590/2590 [00:00<00:00, 11708.86it/s]


In [433]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'COD_LOJA'] == 'VO':
        df_pedido.loc[i, 'NU_CPFCNPJ'] = list(df_cli_ped[(df_cli_ped['CD_LOJA'] == df_pedido.loc[i, 'CD_LOJA']) &
                                             (df_cli_ped['SG_PEDIDO'] == df_pedido.loc[i, 'SERIE']) &
                                             (df_cli_ped['NU_PEDIDO'] == df_pedido.loc[i, 'NU_NOTA'])]['NU_CPFCNPJ'])[0]

100%|██████████| 2590/2590 [00:00<00:00, 40432.22it/s]


In [434]:
df_pedido = pd.merge(df_pedido, df_cliente, how='inner', on='CODCLI')

In [435]:
for i in tqdm(df_pedido.index):

    if df_pedido.loc[i, 'COD_LOJA'] == 'VO':
        identify = df_pedido.loc[i, 'NU_CPFCNPJ']
    else:
        identify = df_pedido.loc[i, 'CPF_CGC']

    if df_pedido.loc[i, 'CODAREA'] == '104':
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CNPJ*'
    elif len(identify) < 14:
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CPF'
    else:
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CNPJ'

100%|██████████| 2590/2590 [00:00<00:00, 9585.97it/s]


## Devolução

In [436]:
df_prod_ent = PostgreSQL.read_postgres(full_query=f'select a.*, (a.VL_PRECO*a.QT_DEVOLVE) as VL_TOTAL_DEV, b.IN_CLIFOR as IN_CLIFOR1, b.IN_CANCELA, UPPER(b.NFEENVSTAT) as NFEENVSTAT, c.FORMA_PGTO from "{_schema}".prod_ent a inner join "{_schema}".entrada b on a.CD_LOJA = b.CD_LOJA and a.SG_SERIE = b.SG_SERIE and a.NU_NOTA = b.NU_NOTA inner join "{_schema}".venda c on a.CD_LOJA = c.CD_LOJA and a.SG_ORIGEM = c.SERIE and a.NU_ORIGEM = c.NU_NOTA where a.DT_EMISSAO >= \'{date_query}\'')

In [437]:
df_prod_ent = df_prod_ent[(df_prod_ent['CD_LOJA'] == '05') &
                          (df_prod_ent['IN_CANCELA'] == 'N') &
                          (df_prod_ent['IN_CLIFOR1'] == 'C') &
                          (df_prod_ent['CD_CFOP'] != '1949') &
                          (df_prod_ent['CD_CFOP'] != '2949') &
                          (df_prod_ent['CD_CFOP'] != '1603') &
                          (df_prod_ent['CD_CLIENTE'] != '13996') &
                          (df_prod_ent['CD_CLIENTE'] != '16100') &
                          (df_prod_ent['CD_CLIENTE'] != '18400') &
                          (df_prod_ent['CD_CLIENTE'] != '20000') &
                          (df_prod_ent['NFEENVSTAT'].str.contains('DENEG') != True)]

In [438]:
for i in tqdm(df_prod_ent.index):
    if df_prod_ent.loc[i, 'FORMA_PGTO'] == 'M':
        df_prod_ent.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_prod_ent.loc[i, 'COD_LOJA'] = df_prod_ent.loc[i, 'CD_LOJA']

100%|██████████| 50/50 [00:00<00:00, 6243.38it/s]


In [439]:
df_prod_ent['VL_TOTAL_DEV'] = df_prod_ent['VL_TOTAL_DEV'].astype(float)

In [440]:
df_prod_ent = df_prod_ent.rename(columns={'CD_CLIENTE':'CODCLI'})
df_prod_ent = pd.merge(df_prod_ent, df_cliente, how='inner', on='CODCLI')

In [441]:
for i in tqdm(df_prod_ent.index):

    identify = df_prod_ent.loc[i, 'CPF_CGC']

    if df_prod_ent.loc[i, 'CODAREA'] == '104':
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CNPJ*'
    elif len(identify) < 14:
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CPF'
    else:
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CNPJ'

100%|██████████| 50/50 [00:00<00:00, 9992.62it/s]


## Metas

In [442]:
df_metas = pd.read_excel('Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\PROJETOS\\EXCEL\\BASES\\DATAS.xlsx', sheet_name='Metas')

In [443]:
df_data = pd.read_excel('Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\PROJETOS\\EXCEL\\BASES\\DATAS.xlsx', sheet_name='Datas')

In [444]:
df_datas = df_data[df_data['Mês - Ano'] == df_data[df_data['Data'] == date_query]['Mês - Ano'].values[0]].reset_index(drop=True)

In [445]:
df_meta_mes = df_metas[df_metas['Mês - Ano'] == df_datas['Mês - Ano'][0]]

In [446]:
df_meta_data = df_datas.drop(columns=['Dia', 'Mês', 'Ano', 'Dia da Semana', 'Nome - Mês', 'Semestre', 'Trimestre', 'Sequencial - Dia da Semana', 'Dias Úteis']).merge(df_meta_mes.drop(columns=['Mês', 'Ano', 'Dias Úteis', 'Qtd Sábados', 'Nome - Mês', 'Loja']), how='inner', on='Mês - Ano')

In [447]:
for i in df_meta_data.index:
    if df_meta_data.loc[i, 'Dia Útil'] == 'Sim':
        df_meta_data.loc[i, 'Meta'] = df_meta_data.loc[i, 'Meta Diária']
    
    elif df_meta_data.loc[i, 'Dia Útil'] == 'Meio':
        df_meta_data.loc[i, 'Meta'] = df_meta_data.loc[i, 'Meta Sábado']

    else:
        df_meta_data.loc[i, 'Meta'] = 0

df_meta_data = df_meta_data.rename(columns={'Cd_Loja':'CD_LOJA', 'Data':'DT_EMISSAO'})

# Cálculo

In [448]:
df_venda_bruta = df_pedido.groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_11804\464804912.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_venda_bruta = df_pedido.groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})


In [449]:
df_devolucao = df_prod_ent.groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_11804\793352684.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_devolucao = df_prod_ent.groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})


In [450]:
df_venda_bruta_cnpj = df_pedido[df_pedido['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})
df_devolucao_cnpj = df_prod_ent[df_prod_ent['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_11804\3541458602.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_venda_bruta_cnpj = df_pedido[df_pedido['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})
C:\Users\admin\AppData\Local\Temp\ipykernel_11804\3541458602.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_devolucao_cnpj = df_prod_ent[df_prod_ent['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})


In [451]:
df_venda_bruta_cpf = df_pedido[df_pedido['TIPO_CLIENTE'] == 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})
df_devolucao_cpf = df_prod_ent[df_prod_ent['TIPO_CLIENTE'] == 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_11804\4099021309.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_venda_bruta_cpf = df_pedido[df_pedido['TIPO_CLIENTE'] == 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})
C:\Users\admin\AppData\Local\Temp\ipykernel_11804\4099021309.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_devolucao_cpf = df_prod_ent[df_prod_ent['TIPO_CLIENTE'] == 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})


In [452]:
df_final = pd.merge(df_venda_bruta, df_devolucao, how='outer', on=['CD_LOJA', 'DT_EMISSAO'])

df_final = df_final.sort_values(by=['DT_EMISSAO', 'CD_LOJA'])
df_final = df_final.reset_index(drop=True)

df_final = pd.merge(df_meta_data, df_final, how='left', on=['CD_LOJA', 'DT_EMISSAO'])

In [453]:
df_final = df_final.fillna(0)

In [454]:
df_hoje = df_final
df_hoje = df_hoje.rename(columns={'VALOR_TOT':'VENDA BRUTA', 'VL_TOTAL_DEV':'DEVOLUÇÃO', 'Meta':'META'})

for i in tqdm(df_hoje.index):
    df_hoje.loc[i, 'VENDA LÍQUIDA'] = (df_hoje.loc[i, 'VENDA BRUTA'] - df_hoje.loc[i, 'DEVOLUÇÃO'])
    
    client_cgc = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] != 'CPF')]['CPF_CGC'].drop_duplicates())

    client_cpf = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] == 'CPF')]['CPF_CGC'])

    venda_cgc = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_pedido['TIPO_CLIENTE'] != 'CPF')]['VALOR_TOT'])

    venda_cpf = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_pedido['TIPO_CLIENTE'] == 'CPF')]['VALOR_TOT'])

    devol_cgc = list(df_prod_ent[(df_prod_ent['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_prod_ent['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_prod_ent['TIPO_CLIENTE'] != 'CPF')]['VL_TOTAL_DEV'])

    devol_cpf = list(df_prod_ent[(df_prod_ent['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_prod_ent['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_prod_ent['TIPO_CLIENTE'] == 'CPF')]['VL_TOTAL_DEV'])

    if sum(devol_cgc) > 0 and sum(venda_cgc) > 0:
        percent_cgc = sum(devol_cgc) / sum(venda_cgc)
    else:
        percent_cgc = 0

    if sum(devol_cpf) > 0 and sum(venda_cpf) > 0:
        percent_cpf = sum(devol_cpf) / sum(venda_cpf)
    else:
        percent_cpf = 0

    df_hoje.loc[i, 'VENDA_CNPJ'] = (sum(venda_cgc) - sum(devol_cgc))
    df_hoje.loc[i, '% DEV CNPJ'] = percent_cgc
    df_hoje.loc[i, 'CNPJ'] = len(client_cgc)

    df_hoje.loc[i, 'VENDA_CPF'] = (sum(venda_cpf) - sum(devol_cpf))
    df_hoje.loc[i, '% DEV CPF'] = percent_cpf
    df_hoje.loc[i, 'CPF'] = len(client_cpf)


    df_hoje.loc[i, '% DEV'] = df_hoje.loc[i, 'DEVOLUÇÃO'] / df_hoje.loc[i, 'VENDA BRUTA']

  0%|          | 0/155 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_11804\3080772628.py:50: RuntimeWarning: invalid value encountered in double_scalars
  df_hoje.loc[i, '% DEV'] = df_hoje.loc[i, 'DEVOLUÇÃO'] / df_hoje.loc[i, 'VENDA BRUTA']
100%|██████████| 155/155 [00:00<00:00, 211.71it/s]


In [455]:
df_hoje = df_hoje.fillna(0)

In [456]:
df_print = df_hoje[df_hoje['CD_LOJA'] == 'VO'].reset_index(drop=True)
df_print[['DT_EMISSAO',
          'Nome - Dia da Semana',
          'CD_LOJA',
          'META',
          'VENDA BRUTA',
          'VENDA LÍQUIDA',
          'DEVOLUÇÃO',
          '% DEV',
          'VENDA_CNPJ',
          '% DEV CNPJ',
          'CNPJ',
          'VENDA_CPF',
          '% DEV CPF',
          'CPF']]

,DT_EMISSAO,Nome - Dia da Semana,CD_LOJA,META,VENDA BRUTA,VENDA LÍQUIDA,DEVOLUÇÃO,% DEV,VENDA_CNPJ,% DEV CNPJ,CNPJ,VENDA_CPF,% DEV CPF,CPF
0,2022-12-01,Quinta-feira,VO,19047.619048,12246.23,12246.23,0.0,0.0,600.6,0.0,1.0,11645.63,0.0,79.0
1,2022-12-02,Sexta-feira,VO,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,2022-12-03,Sábado,VO,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,2022-12-04,Domingo,VO,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,2022-12-05,Segunda-feira,VO,19047.619048,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
5,2022-12-06,Terça-feira,VO,19047.619048,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
6,2022-12-07,Quarta-feira,VO,19047.619048,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
7,2022-12-08,Quinta-feira,VO,19047.619048,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
8,2022-12-09,Sexta-feira,VO,19047.619048,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
9,2022-12-10,Sábado,VO,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [457]:
total_meta  = float(sum(list(df_print['META'])))
total_bruto = float(sum(list(df_print['VENDA BRUTA'])))
total_liqui = float(sum(list(df_print['VENDA LÍQUIDA'])))
total_devol = float(sum(list(df_print['DEVOLUÇÃO'])))

total_venda_pj = float(sum(list(df_print['VENDA_CNPJ'])))
total_venda_pf = float(sum(list(df_print['VENDA_CPF'])))


percent_total_dev = (total_devol / total_bruto)